# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f05049e9d90>
├── 'a' --> tensor([[-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471]])
└── 'x' --> <FastTreeValue 0x7f05049e9970>
    └── 'c' --> tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                        [ 0.3865, -0.3010, -0.0750,  0.8125],
                        [-0.1364, -0.3591,  0.1179,  0.1970]])

In [4]:
t.a

tensor([[-0.1412,  1.3471, -1.1001],
        [ 0.1245, -0.0482,  0.3471]])

In [5]:
%timeit t.a

66.1 ns ± 0.0593 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f05049e9d90>
├── 'a' --> tensor([[ 0.8380,  0.0300, -0.5472],
│                   [ 0.1472, -2.6357, -0.2420]])
└── 'x' --> <FastTreeValue 0x7f05049e9970>
    └── 'c' --> tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                        [ 0.3865, -0.3010, -0.0750,  0.8125],
                        [-0.1364, -0.3591,  0.1179,  0.1970]])

In [7]:
%timeit t.a = new_value

73.5 ns ± 0.0544 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471]]),
    x: Batch(
           c: tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                      [ 0.3865, -0.3010, -0.0750,  0.8125],
                      [-0.1364, -0.3591,  0.1179,  0.1970]]),
       ),
)

In [10]:
b.a

tensor([[-0.1412,  1.3471, -1.1001],
        [ 0.1245, -0.0482,  0.3471]])

In [11]:
%timeit b.a

60.2 ns ± 0.0199 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5728,  0.1775, -0.4862],
               [ 0.3594,  0.8024, -0.6150]]),
    x: Batch(
           c: tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                      [ 0.3865, -0.3010, -0.0750,  0.8125],
                      [-0.1364, -0.3591,  0.1179,  0.1970]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.0884 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.566 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 37.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

147 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 533 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f044ec81dc0>
├── 'a' --> tensor([[[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]],
│           
│                   [[-0.1412,  1.3471, -1.1001],
│                    [ 0.1245, -0.0482,  0.3471]]])
└── 'x' --> <FastTreeValue 0x7f044ec81fd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 72.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f044ec81c10>
├── 'a' --> tensor([[-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471],
│                   [-0.1412,  1.3471, -1.1001],
│                   [ 0.1245, -0.0482,  0.3471]])
└── 'x' --> <FastTreeValue 0x7f0508a761f0>
    └── 'c' --> tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                        [ 0.3865, -0.3010, -0.0750,  0.8125],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 55.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.9 µs ± 51.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]],
       
               [[-0.1412,  1.3471, -1.1001],
                [ 0.1245, -0.0482,  0.3471]]]),
    x: Batch(
           c: tensor([[[-0.2984, -0.1797, -0.1971,  1.1798],
                       [ 0.3865, -0.3010, -0.0750,  0.8125],
                       [-0.1364, -0.3591,  0.1179,  0.1970]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471],
               [-0.1412,  1.3471, -1.1001],
               [ 0.1245, -0.0482,  0.3471]]),
    x: Batch(
           c: tensor([[-0.2984, -0.1797, -0.1971,  1.1798],
                      [ 0.3865, -0.3010, -0.0750,  0.8125],
                      [-0.1364, -0.3591,  0.1179,  0.1970],
                      [-0.2984, -0.1797, -0.1971,  1.1798],
                      [ 0.3865, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 2.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
